# Crawl Redeemer of Israel from the Church of Jesus Christ of Latter-day Saints

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from urllib.parse import urljoin, urlparse

from bs4 import BeautifulSoup
import requests
import time

from models.crawl_utils import get_page, save_page

In [ ]:
# config
host = 'https://www.redeemerofisrael.org/sitemap.xml'
base_dir = '../data/raw/redeemer_of_israel'
delay_seconds = 5

if not os.path.exists(base_dir):
    os.makedirs(base_dir)


In [ ]:
def extract_links(xml_content):
    #Parse the XML content using BeautifulSoup
    soup = BeautifulSoup(xml_content, 'xml')

    #Find all <loc> tags in the XML
    loc_tags = soup.find_all('loc')

    #Extract and store the text inside each <loc> tag
    loc_texts = [loc.get_text() for loc in loc_tags]

    return loc_texts

In [ ]:
def get_path(url):
    path_components = urlparse(url).path.split('/')
    path = path_components[-1]
    if path.endswith('.html'):
        path = path[:-5]
    # print(path_components)
    return os.path.join(base_dir, f"{path}.json")

In [ ]:
def get_sitemap(host):
    headers = {
        'Content-Type': 'text/xml',
    }
    response = requests.get(host, headers=headers)
    return response

In [ ]:
response = get_sitemap(host)
if response.status_code != 200:
    print(f"ERROR {response.status_code}")
xml_content = response.content

In [ ]:
xml_links = extract_links(xml_content)
print(len(xml_links))
print(xml_links[2])

In [ ]:
for url in xml_links:
    path_file =  get_path(url)
    print(path_file)
    if os.path.exists(path_file):
        continue
    status_code, html = get_page(url, delay_seconds)
    if status_code != 200:
        print("Error!", status_code , url)
        continue
    save_page(path_file,url,html)
    
print("End")